<a href="https://colab.research.google.com/github/dishadtu/Image_Classification_CIFAR_Tensorflow_Keras/blob/main/Image_Classification_CIFAR_Tensorflow_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os
#https://www.scaler.com/topics/deep-learning/tensorflow-image-classification/

train_dataset, validation_dataset, test_dataset = tfds.load(
    "cifar10",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,
)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete3CTI58/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete3CTI58/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
print("Number of training samples: %d" % tf.data.experimental.cardinality(train_dataset))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_dataset)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_dataset))


Number of training samples: 20000
Number of validation samples: 5000
Number of test samples: 5000


In [ ]:
size = (128, 128)
bs = 64

train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
test_dataset = test_dataset.map(lambda x, y: (tf.image.resize(x, size), y))

train_dataset = train_dataset.cache().batch(bs).prefetch(buffer_size=10)
validation_dataset = validation_dataset.cache().batch(bs).prefetch(buffer_size=10)
test_dataset = test_dataset.cache().batch(bs).prefetch(buffer_size=10)


In [ ]:
aug_transforms = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

model_base = keras.applications.ResNet50(
    weights=None,
    input_shape=(128, 128, 3),
    classes = 10,
    include_top=True
)

inputs = keras.Input(shape=(128, 128, 3))

x = aug_transforms(inputs)
x = model_base(x, training=False)
outputs = keras.layers.Dense(10)(x)
final_model = keras.Model(inputs, outputs)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 10)                23608202  
                                                                 
 dense (Dense)               (None, 10)                110       
                                                                 
Total params: 23,608,312
Trainable params: 23,555,192
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
final_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

num_epochs = 2
final_model.fit(train_dataset, epochs=num_epochs, validation_data=validation_dataset)


Epoch 1/5
313/313 [==============================] - 3986s 13s/step - loss: 2.3196 - sparse_categorical_accuracy: 0.1028 - val_loss: 2.3065 - val_sparse_categorical_accuracy: 0.0972
Epoch 2/5


In [ ]:
results = final_model.evaluate(test_dataset, batch_size=128)
print("test loss, test acc:", results)


NameError: ignored

Testing a single image

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform

#cifar10 labels
list_labels = ["airplane","automobile","bird","cat","deer","dog", "frog", "horse", "ship", "truck"]
dict_labels = {i: list_labels[i] for i in range(len(list_labels))}

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (128, 128, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

im = Image.open('dog.jpg').resize((128,128));print(dict_labels[int(np.argmax(final_model.predict(image)))]); im
